# 데이터 수집
dataframe에 데이터 가져오기

In [60]:
import os
import pandas as pd

numbers_path = 'data/숫자인식/'
alphabets_path = 'data/알파벳인식/'
free_patterns_path = 'data/자유패턴/'

number_classes = [str(i) for i in range(10)]
alphabet_classes = [chr(i) for i in range(ord('A'), ord('Z')+1)]

number_folders = {}
for class_name in number_classes:
    number_folders[class_name] = '0' + class_name

data = []

for number, folder_name in number_folders.items():
    n_folder_path = os.path.join(numbers_path, folder_name, 'n'+folder_name)
    r_folder_path = os.path.join(numbers_path, folder_name, 'r_n'+folder_name)
    for file in os.listdir(n_folder_path):
        data.append({
            'image_path': os.path.join(n_folder_path, file),
            'category': 'number',
            'target': number,
            'r': False
        })
    for file in os.listdir(r_folder_path):
        data.append({
            'image_path': os.path.join(r_folder_path, file),
            'category': 'number',
            'target': number,
            'r': True
        })

for letter in alphabet_classes:
    n_folder_path = os.path.join(alphabets_path, letter, letter)
    r_folder_path = os.path.join(alphabets_path, letter, 'r_'+letter)
    for file in os.listdir(n_folder_path):
        data.append({
            'image_path': os.path.join(n_folder_path, file),
            'category': 'alphabet',
            'target': letter,
            'r': False
        })
    for file in os.listdir(r_folder_path):
        data.append({
            'image_path': os.path.join(r_folder_path, file),
            'category': 'alphabet',
            'target': letter,
            'r': True
        })

for root, dirs, files in os.walk(free_patterns_path):
    for file in files:
        data.append({
            'image_path': os.path.join(root, file),
            'category': 'free_pattern',
            'target': None,
            'r': None
        })


df = pd.DataFrame(data)
df.sample(10)

,image_path,category,target,r
1174,data/알파벳인식/B/r_B/4d_al_r_b_24.jpg,alphabet,B,True
2052,data/알파벳인식/K/K/4d_al_K_32.jpg,alphabet,K,False
2348,data/알파벳인식/N/N/4d_al_N_28.jpg,alphabet,N,False
3917,data/자유패턴/30/30_r/30_r10.jpg,free_pattern,None,None
910,data/숫자인식/09/n09/4d_n09_10.jpg,number,9,False
3839,data/자유패턴/20/20_r/20_r10.jpg,free_pattern,None,None
941,data/숫자인식/09/n09/4d_n09_42.jpg,number,9,False
2561,data/알파벳인식/P/P/4d_al_P_41.jpg,alphabet,P,False
1604,data/알파벳인식/F/r_F/4d_al_r_F_49.jpg,alphabet,F,True
3006,data/알파벳인식/T/r_T/4d_al_r_T_36.jpg,alphabet,T,True


In [ ]:
numbers_test_path = 'data/숫자인식/'
alphabets_path = 'data/알파벳인식/'
free_patterns_path = 'data/자유패턴/'

number_classes = [str(i) for i in range(10)]
alphabet_classes = [chr(i) for i in range(ord('A'), ord('Z')+1)]

number_folders = {}
for class_name in number_classes:
    number_folders[class_name] = '0' + class_name

data = []

for number, folder_name in number_folders.items():
    n_folder_path = os.path.join(numbers_path, folder_name, 'n'+folder_name)
    r_folder_path = os.path.join(numbers_path, folder_name, 'r_n'+folder_name)
    for file in os.listdir(n_folder_path):
        data.append({
            'image_path': os.path.join(n_folder_path, file),
            'category': 'number',
            'target': number,
            'r': False
        })
    for file in os.listdir(r_folder_path):
        data.append({
            'image_path': os.path.join(r_folder_path, file),
            'category': 'number',
            'target': number,
            'r': True
        })

for letter in alphabet_classes:
    n_folder_path = os.path.join(alphabets_path, letter, letter)
    r_folder_path = os.path.join(alphabets_path, letter, 'r_'+letter)
    for file in os.listdir(n_folder_path):
        data.append({
            'image_path': os.path.join(n_folder_path, file),
            'category': 'alphabet',
            'target': letter,
            'r': False
        })
    for file in os.listdir(r_folder_path):
        data.append({
            'image_path': os.path.join(r_folder_path, file),
            'category': 'alphabet',
            'target': letter,
            'r': True
        })

for root, dirs, files in os.walk(free_patterns_path):
    for file in files:
        data.append({
            'image_path': os.path.join(root, file),
            'category': 'free_pattern',
            'target': None,
            'r': None
        })


df = pd.DataFrame(data)
df.sample(10)

# 데이터 전처리
- 이미지 전처리 함수 실행
- 분류 원핫 인코딩

In [2]:
import cv2
import numpy as np
from preprocessing import preprocess_img
from sklearn.model_selection import train_test_split

X, y = df['image_path'], df[['category', 'target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.05)

X_train_pp = X_train.apply(preprocess_img)
X_test_pp = X_test.apply(preprocess_img)

y_train_3 = y_train['category']
y_train_3 = pd.get_dummies(y_train_3)
y_train_3 = y_train_3[['number', 'alphabet', 'free_pattern']]

y_test_3 = y_test['category']
y_test_3 = pd.get_dummies(y_test_3)
y_test_3 = y_test_3[['number', 'alphabet', 'free_pattern']]


In [42]:
X_train.size, X_test.size, y_train_3.size, y_test_3.size

(3934, 208, 11802, 624)

# 모델 선택

## 숫자/알파벳/자유패턴 모델

### CNN 조정할 매개변수 
- 이미지 사이즈: 32x32x3, 64x64x3, 224x224x3
- 커널 크기: (3, 3), (5, 5)
- 필터 수: 32, 64
- 숨겨진 단위: 128, 256

In [3]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

# model generator for grid search
def create_model(image_size=(64, 64, 3), kernel_size=(3, 3), filters=32, hidden_units=128, output_size=3):
    model = Sequential([
        Conv2D(filters, kernel_size, activation='relu', input_shape=image_size),
        MaxPooling2D((2, 2)),
        Conv2D(filters*2, kernel_size, activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(hidden_units, activation='relu'),
        Dense(output_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

2023-08-31 10:22:24.907383: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 10:22:24.948002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 10:22:24.948403: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 10:22:25.872629: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


가장 간단한 모델부터

In [10]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(3, 3), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)

def s_preprocess_img(image, img_size=(244, 244)):
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    image = cv2.resize(image, img_size)
    image = image / 255.0
    return image

X_train_32 = X_train.apply(lambda x: s_preprocess_img(x, img_size=(32, 32)))
X_train_32 = np.stack(X_train_32.values)

model.fit(X_train_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50


2023-08-31 07:13:46.781627: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30523392 exceeds 10% of free system memory.


78/78 [==============================] - 3s 33ms/step - loss: 0.5520 - accuracy: 0.7528 - val_loss: 0.4196 - val_accuracy: 0.8071
Epoch 2/50
78/78 [==============================] - 2s 31ms/step - loss: 0.3571 - accuracy: 0.8426 - val_loss: 0.3387 - val_accuracy: 0.8376
Epoch 3/50
78/78 [==============================] - 2s 31ms/step - loss: 0.3132 - accuracy: 0.8603 - val_loss: 0.3239 - val_accuracy: 0.8682
Epoch 4/50
78/78 [==============================] - 2s 31ms/step - loss: 0.2817 - accuracy: 0.8885 - val_loss: 0.2943 - val_accuracy: 0.8601
Epoch 5/50
78/78 [==============================] - 2s 31ms/step - loss: 0.2498 - accuracy: 0.8949 - val_loss: 0.3225 - val_accuracy: 0.8505
Epoch 6/50
78/78 [==============================] - 2s 31ms/step - loss: 0.2328 - accuracy: 0.8981 - val_loss: 0.2848 - val_accuracy: 0.8762
Epoch 7/50
78/78 [==============================] - 2s 30ms/step - loss: 0.2091 - accuracy: 0.9118 - val_loss: 0.2889 - val_accuracy: 0.8875
Epoch 8/50
78/78 [======

In [12]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(3, 3), 
    filters=32, 
    hidden_units=128, 
    output_size=3
)

model.fit(X_train_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 2s 15ms/step - loss: 0.6334 - accuracy: 0.7186 - val_loss: 0.5941 - val_accuracy: 0.5932
Epoch 2/50
78/78 [==============================] - 1s 13ms/step - loss: 0.4016 - accuracy: 0.8164 - val_loss: 0.3666 - val_accuracy: 0.8392
Epoch 3/50
78/78 [==============================] - 1s 13ms/step - loss: 0.3218 - accuracy: 0.8583 - val_loss: 0.3548 - val_accuracy: 0.8312
Epoch 4/50
78/78 [==============================] - 1s 13ms/step - loss: 0.2914 - accuracy: 0.8764 - val_loss: 0.3387 - val_accuracy: 0.8457
Epoch 5/50
78/78 [==============================] - 1s 13ms/step - loss: 0.2798 - accuracy: 0.8800 - val_loss: 0.3811 - val_accuracy: 0.8328
Epoch 6/50
78/78 [==============================] - 1s 13ms/step - loss: 0.2481 - accuracy: 0.8973 - val_loss: 0.4266 - val_accuracy: 0.7990
Epoch 7/50
78/78 [==============================] - 1s 13ms/step - loss: 0.2533 - accuracy: 0.8917 - val_loss: 0.2819 - val_accuracy: 0.8746
Epoch 8/50
78

In [6]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(3, 3), 
    filters=32, 
    hidden_units=128, 
    output_size=3
)
X_train_pp_32 = X_train_pp.apply(cv2.resize, dsize=(32, 32))
X_train_pp_32 = np.stack(X_train_pp_32.values)
model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 3s 22ms/step - loss: 0.7143 - accuracy: 0.6828 - val_loss: 0.5595 - val_accuracy: 0.7347
Epoch 2/50
78/78 [==============================] - 1s 16ms/step - loss: 0.5053 - accuracy: 0.7605 - val_loss: 0.4962 - val_accuracy: 0.7605
Epoch 3/50
78/78 [==============================] - 1s 16ms/step - loss: 0.4343 - accuracy: 0.7830 - val_loss: 0.4365 - val_accuracy: 0.7910
Epoch 4/50
78/78 [==============================] - 1s 15ms/step - loss: 0.3910 - accuracy: 0.8104 - val_loss: 0.4613 - val_accuracy: 0.7637
Epoch 5/50
78/78 [==============================] - 1s 15ms/step - loss: 0.3496 - accuracy: 0.8225 - val_loss: 0.3721 - val_accuracy: 0.8183
Epoch 6/50
78/78 [==============================] - 1s 18ms/step - loss: 0.2931 - accuracy: 0.8684 - val_loss: 0.3429 - val_accuracy: 0.8248
Epoch 7/50
78/78 [==============================] - 1s 19ms/step - loss: 0.2644 - accuracy: 0.8788 - val_loss: 0.3260 - val_accuracy: 0.8473
Epoch 8/50
78

In [7]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(5, 5), 
    filters=32, 
    hidden_units=128, 
    output_size=3
)

model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 3s 20ms/step - loss: 0.7557 - accuracy: 0.6828 - val_loss: 0.6037 - val_accuracy: 0.7010
Epoch 2/50
78/78 [==============================] - 1s 18ms/step - loss: 0.5226 - accuracy: 0.7649 - val_loss: 0.4762 - val_accuracy: 0.7588
Epoch 3/50
78/78 [==============================] - 1s 19ms/step - loss: 0.4526 - accuracy: 0.7713 - val_loss: 0.4402 - val_accuracy: 0.7781
Epoch 4/50
78/78 [==============================] - 2s 21ms/step - loss: 0.4237 - accuracy: 0.7923 - val_loss: 0.4141 - val_accuracy: 0.8006
Epoch 5/50
78/78 [==============================] - 2s 22ms/step - loss: 0.3814 - accuracy: 0.8140 - val_loss: 0.3767 - val_accuracy: 0.8199
Epoch 6/50
78/78 [==============================] - 2s 21ms/step - loss: 0.3565 - accuracy: 0.8297 - val_loss: 0.3595 - val_accuracy: 0.8376
Epoch 7/50
78/78 [==============================] - 2s 19ms/step - loss: 0.3098 - accuracy: 0.8490 - val_loss: 0.3322 - val_accuracy: 0.8441
Epoch 8/50
78

In [8]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(3, 3), 
    filters=64, 
    hidden_units=128, 
    output_size=3
)

model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 4s 35ms/step - loss: 0.7590 - accuracy: 0.6767 - val_loss: 0.5754 - val_accuracy: 0.7331
Epoch 2/50
78/78 [==============================] - 3s 37ms/step - loss: 0.5082 - accuracy: 0.7516 - val_loss: 0.4777 - val_accuracy: 0.7540
Epoch 3/50
78/78 [==============================] - 3s 33ms/step - loss: 0.4645 - accuracy: 0.7649 - val_loss: 0.4775 - val_accuracy: 0.7540
Epoch 4/50
78/78 [==============================] - 2s 31ms/step - loss: 0.4317 - accuracy: 0.7826 - val_loss: 0.4146 - val_accuracy: 0.8023
Epoch 5/50
78/78 [==============================] - 3s 33ms/step - loss: 0.3850 - accuracy: 0.8136 - val_loss: 0.3864 - val_accuracy: 0.8006
Epoch 6/50
78/78 [==============================] - 3s 36ms/step - loss: 0.3364 - accuracy: 0.8390 - val_loss: 0.3401 - val_accuracy: 0.8248
Epoch 7/50
78/78 [==============================] - 3s 33ms/step - loss: 0.2900 - accuracy: 0.8663 - val_loss: 0.3191 - val_accuracy: 0.8376
Epoch 8/50
78

In [9]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(3, 3), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)

model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 4s 38ms/step - loss: 0.7142 - accuracy: 0.6884 - val_loss: 0.6161 - val_accuracy: 0.7058
Epoch 2/50
78/78 [==============================] - 3s 38ms/step - loss: 0.5151 - accuracy: 0.7488 - val_loss: 0.4950 - val_accuracy: 0.7524
Epoch 3/50
78/78 [==============================] - 3s 43ms/step - loss: 0.4520 - accuracy: 0.7814 - val_loss: 0.4336 - val_accuracy: 0.7846
Epoch 4/50
78/78 [==============================] - 3s 37ms/step - loss: 0.4122 - accuracy: 0.8043 - val_loss: 0.4153 - val_accuracy: 0.7781
Epoch 5/50
78/78 [==============================] - 3s 38ms/step - loss: 0.3803 - accuracy: 0.8039 - val_loss: 0.3757 - val_accuracy: 0.7974
Epoch 6/50
78/78 [==============================] - 3s 44ms/step - loss: 0.3301 - accuracy: 0.8426 - val_loss: 0.3573 - val_accuracy: 0.8199
Epoch 7/50
78/78 [==============================] - 3s 38ms/step - loss: 0.3109 - accuracy: 0.8474 - val_loss: 0.3374 - val_accuracy: 0.8408
Epoch 8/50
78

In [10]:
model = create_model(
    image_size=(32, 32, 3), 
    kernel_size=(5, 5), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)

model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 5s 55ms/step - loss: 0.7613 - accuracy: 0.6695 - val_loss: 0.7454 - val_accuracy: 0.6768
Epoch 2/50
78/78 [==============================] - 4s 52ms/step - loss: 0.5210 - accuracy: 0.7456 - val_loss: 0.4782 - val_accuracy: 0.7814
Epoch 3/50
78/78 [==============================] - 4s 53ms/step - loss: 0.4415 - accuracy: 0.7838 - val_loss: 0.4437 - val_accuracy: 0.7733
Epoch 4/50
78/78 [==============================] - 4s 51ms/step - loss: 0.4228 - accuracy: 0.7939 - val_loss: 0.4386 - val_accuracy: 0.7846
Epoch 5/50
78/78 [==============================] - 4s 49ms/step - loss: 0.3660 - accuracy: 0.8269 - val_loss: 0.3680 - val_accuracy: 0.8167
Epoch 6/50
78/78 [==============================] - 4s 50ms/step - loss: 0.3149 - accuracy: 0.8498 - val_loss: 0.4632 - val_accuracy: 0.7781
Epoch 7/50
78/78 [==============================] - 4s 52ms/step - loss: 0.2765 - accuracy: 0.8712 - val_loss: 0.5289 - val_accuracy: 0.7910
Epoch 8/50
78

In [ ]:
model = create_model(
    image_size=(64, 64, 3), 
    kernel_size=(3, 3), 
    filters=32, 
    hidden_units=128, 
    output_size=3
)

X_train_64 = X_train.apply(lambda x: s_preprocess_img(x, img_size=(64, 64)))

In [15]:
X_train_64 = np.stack(X_train_64)

model.fit(X_train_64, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50


2023-08-31 02:11:31.058191: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122093568 exceeds 10% of free system memory.


78/78 [==============================] - 5s 60ms/step - loss: 0.5178 - accuracy: 0.7790 - val_loss: 0.3694 - val_accuracy: 0.8441
Epoch 2/50
78/78 [==============================] - 4s 58ms/step - loss: 0.3380 - accuracy: 0.8523 - val_loss: 0.3611 - val_accuracy: 0.8553
Epoch 3/50
78/78 [==============================] - 4s 55ms/step - loss: 0.2803 - accuracy: 0.8829 - val_loss: 0.2945 - val_accuracy: 0.8698
Epoch 4/50
78/78 [==============================] - 4s 56ms/step - loss: 0.2603 - accuracy: 0.8877 - val_loss: 0.3023 - val_accuracy: 0.8875
Epoch 5/50
78/78 [==============================] - 4s 55ms/step - loss: 0.2374 - accuracy: 0.9034 - val_loss: 0.2755 - val_accuracy: 0.8826
Epoch 6/50
78/78 [==============================] - 4s 55ms/step - loss: 0.2096 - accuracy: 0.9130 - val_loss: 0.2635 - val_accuracy: 0.9035
Epoch 7/50
78/78 [==============================] - 4s 55ms/step - loss: 0.1771 - accuracy: 0.9275 - val_loss: 0.3222 - val_accuracy: 0.8666
Epoch 8/50
78/78 [======

In [16]:
model = create_model(
    image_size=(64, 64, 3), 
    kernel_size=(5, 5), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)

model.fit(X_train_64, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50


2023-08-31 02:12:42.477785: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122093568 exceeds 10% of free system memory.


78/78 [==============================] - 19s 228ms/step - loss: 0.6915 - accuracy: 0.7222 - val_loss: 0.4370 - val_accuracy: 0.7990
Epoch 2/50
78/78 [==============================] - 18s 232ms/step - loss: 0.3641 - accuracy: 0.8422 - val_loss: 0.3887 - val_accuracy: 0.8215
Epoch 3/50
78/78 [==============================] - 17s 218ms/step - loss: 0.3023 - accuracy: 0.8712 - val_loss: 0.3236 - val_accuracy: 0.8617
Epoch 4/50
78/78 [==============================] - 17s 215ms/step - loss: 0.2713 - accuracy: 0.8861 - val_loss: 0.2899 - val_accuracy: 0.8746
Epoch 5/50
78/78 [==============================] - 17s 222ms/step - loss: 0.2709 - accuracy: 0.8829 - val_loss: 0.2858 - val_accuracy: 0.8762
Epoch 6/50
78/78 [==============================] - 18s 231ms/step - loss: 0.2395 - accuracy: 0.8969 - val_loss: 0.2730 - val_accuracy: 0.8714
Epoch 7/50
78/78 [==============================] - 17s 218ms/step - loss: 0.2214 - accuracy: 0.9034 - val_loss: 0.2526 - val_accuracy: 0.8939
Epoch 8/50

In [18]:
model = create_model(
    image_size=(64, 64, 3), 
    kernel_size=(3, 3), 
    filters=32, 
    hidden_units=128, 
    output_size=3
)


X_train_pp_64 = X_train_pp.apply(lambda x: cv2.resize(x, dsize=(64, 64)))
X_train_pp_64 = np.stack(X_train_pp_64.values)

model.fit(X_train_pp_64, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50


2023-08-31 02:17:35.577256: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122093568 exceeds 10% of free system memory.


78/78 [==============================] - 5s 56ms/step - loss: 0.6945 - accuracy: 0.6844 - val_loss: 0.5080 - val_accuracy: 0.7572
Epoch 2/50
78/78 [==============================] - 4s 56ms/step - loss: 0.4492 - accuracy: 0.7738 - val_loss: 0.3924 - val_accuracy: 0.8006
Epoch 3/50
78/78 [==============================] - 4s 53ms/step - loss: 0.4085 - accuracy: 0.7987 - val_loss: 0.4029 - val_accuracy: 0.7878
Epoch 4/50
78/78 [==============================] - 4s 55ms/step - loss: 0.3381 - accuracy: 0.8402 - val_loss: 0.4666 - val_accuracy: 0.7765
Epoch 5/50
78/78 [==============================] - 5s 59ms/step - loss: 0.2921 - accuracy: 0.8724 - val_loss: 0.2908 - val_accuracy: 0.8537
Epoch 6/50
78/78 [==============================] - 4s 55ms/step - loss: 0.2626 - accuracy: 0.8772 - val_loss: 0.2882 - val_accuracy: 0.8714
Epoch 7/50
78/78 [==============================] - 4s 53ms/step - loss: 0.2242 - accuracy: 0.9058 - val_loss: 0.2473 - val_accuracy: 0.8891
Epoch 8/50
78/78 [======

In [22]:
model = create_model(
    image_size=(64, 64, 3), 
    kernel_size=(5, 5), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)


X_train_pp_64 = X_train_pp.apply(lambda x: cv2.resize(x, dsize=(64, 64)))
X_train_pp_64 = np.stack(X_train_pp_64.values)

model.fit(X_train_pp_64, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 17s 214ms/step - loss: 0.9125 - accuracy: 0.6473 - val_loss: 0.7610 - val_accuracy: 0.6415
Epoch 2/50
78/78 [==============================] - 16s 206ms/step - loss: 0.5541 - accuracy: 0.7456 - val_loss: 0.5275 - val_accuracy: 0.7540
Epoch 3/50
78/78 [==============================] - 16s 208ms/step - loss: 0.5042 - accuracy: 0.7552 - val_loss: 0.5388 - val_accuracy: 0.7331
Epoch 4/50
78/78 [==============================] - 16s 203ms/step - loss: 0.4832 - accuracy: 0.7556 - val_loss: 0.4817 - val_accuracy: 0.7669
Epoch 5/50
78/78 [==============================] - 16s 204ms/step - loss: 0.4280 - accuracy: 0.7770 - val_loss: 0.4564 - val_accuracy: 0.7733
Epoch 6/50
78/78 [==============================] - 17s 212ms/step - loss: 0.4053 - accuracy: 0.8011 - val_loss: 0.4512 - val_accuracy: 0.7910
Epoch 7/50
78/78 [==============================] - 16s 204ms/step - loss: 0.3788 - accuracy: 0.8039 - val_loss: 0.3866 - val_accuracy: 0.8006

In [11]:
model = create_model(
    image_size=(224, 224, 3), 
    kernel_size=(5, 5), 
    filters=64, 
    hidden_units=256, 
    output_size=3
)


X_train_pp_224 = np.stack(X_train_pp.values)

model.fit(X_train_pp_224, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

2023-08-31 07:15:09.679119: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 368181248 exceeds 10% of free system memory.


Epoch 1/50
78/78 [==============================] - 313s 4s/step - loss: 1.2597 - accuracy: 0.6574 - val_loss: 0.8172 - val_accuracy: 0.7412
Epoch 2/50
78/78 [==============================] - 313s 4s/step - loss: 1.0100 - accuracy: 0.7226 - val_loss: 0.8504 - val_accuracy: 0.7283
Epoch 3/50
78/78 [==============================] - 319s 4s/step - loss: 1.0612 - accuracy: 0.7367 - val_loss: 0.5422 - val_accuracy: 0.7653
Epoch 4/50
78/78 [==============================] - 266s 3s/step - loss: 0.9714 - accuracy: 0.7343 - val_loss: 0.9316 - val_accuracy: 0.6318
Epoch 5/50
78/78 [==============================] - 256s 3s/step - loss: 0.7451 - accuracy: 0.6969 - val_loss: 0.9040 - val_accuracy: 0.6270
Epoch 6/50
78/78 [==============================] - 251s 3s/step - loss: 0.9414 - accuracy: 0.6514 - val_loss: 0.9047 - val_accuracy: 0.6318
Epoch 7/50
78/78 [==============================] - 244s 3s/step - loss: 0.8857 - accuracy: 0.6417 - val_loss: 0.9009 - val_accuracy: 0.6302
Epoch 8/50
78

In [12]:
from tensorflow.keras.layers import Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_224, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 64s 811ms/step - loss: 0.7421 - accuracy: 0.7089 - val_loss: 0.4951 - val_accuracy: 0.7685
Epoch 2/50
78/78 [==============================] - 63s 804ms/step - loss: 0.4215 - accuracy: 0.7999 - val_loss: 0.3682 - val_accuracy: 0.8135
Epoch 3/50
78/78 [==============================] - 63s 806ms/step - loss: 0.3144 - accuracy: 0.8571 - val_loss: 0.2959 - val_accuracy: 0.8714
Epoch 4/50
78/78 [==============================] - 63s 805ms/step - loss: 0.2426 - accuracy: 0.8925 - val_loss: 0.2274 - val_accuracy: 0.9019
Epoch 5/50
78/78 [==============================] - 63s 804ms/step - loss: 0.2495 - accuracy: 0.8957 - val_loss: 0.2508 - val_accuracy: 0.9068
Epoch 6/50
78/78 [==============================] - 63s 805ms/step - loss: 0.1763 - accuracy: 0.9312 - val_loss: 0.2541 - val_accuracy: 0.9003
Epoch 7/50
78/78 [==============================] - 63s 806ms/step - loss: 0.1410 - accuracy: 0.9497 - val_loss: 0.1892 - val_accuracy: 0.9148

In [29]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 2s 15ms/step - loss: 0.7358 - accuracy: 0.6844 - val_loss: 0.6248 - val_accuracy: 0.7460
Epoch 2/50
78/78 [==============================] - 1s 16ms/step - loss: 0.5390 - accuracy: 0.7436 - val_loss: 0.5273 - val_accuracy: 0.7460
Epoch 3/50
78/78 [==============================] - 1s 14ms/step - loss: 0.4929 - accuracy: 0.7641 - val_loss: 0.5447 - val_accuracy: 0.7444
Epoch 4/50
78/78 [==============================] - 1s 13ms/step - loss: 0.4539 - accuracy: 0.7705 - val_loss: 0.5234 - val_accuracy: 0.7540
Epoch 5/50
78/78 [==============================] - 1s 14ms/step - loss: 0.4309 - accuracy: 0.7862 - val_loss: 0.4237 - val_accuracy: 0.7717
Epoch 6/50
78/78 [==============================] - 1s 14ms/step - loss: 0.3904 - accuracy: 0.8104 - val_loss: 0.4282 - val_accuracy: 0.7685
Epoch 7/50
78/78 [==============================] - 1s 14ms/step - loss: 0.3604 - accuracy: 0.8237 - val_loss: 0.3667 - val_accuracy: 0.8167
Epoch 8/50
78

In [30]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
78/78 [==============================] - 2s 15ms/step - loss: 0.8167 - accuracy: 0.6453 - val_loss: 0.6200 - val_accuracy: 0.7074
Epoch 2/50
78/78 [==============================] - 1s 14ms/step - loss: 0.5561 - accuracy: 0.7315 - val_loss: 0.5006 - val_accuracy: 0.7460
Epoch 3/50
78/78 [==============================] - 1s 12ms/step - loss: 0.4728 - accuracy: 0.7597 - val_loss: 0.4709 - val_accuracy: 0.7524
Epoch 4/50
78/78 [==============================] - 1s 13ms/step - loss: 0.4257 - accuracy: 0.7979 - val_loss: 0.3977 - val_accuracy: 0.7926
Epoch 5/50
78/78 [==============================] - 1s 13ms/step - loss: 0.3779 - accuracy: 0.8213 - val_loss: 0.3493 - val_accuracy: 0.8215
Epoch 6/50
78/78 [==============================] - 1s 12ms/step - loss: 0.3236 - accuracy: 0.8543 - val_loss: 0.4422 - val_accuracy: 0.7797
Epoch 7/50
78/78 [==============================] - 1s 12ms/step - loss: 0.2829 - accuracy: 0.8752 - val_loss: 0.2939 - val_accuracy: 0.8633
Epoch 8/50
78

In [43]:
y_train_3.value_counts()

number  alphabet  free_pattern
False   True      False           1965
True    False     False            750
False   False     True             391
Name: count, dtype: int64

In [52]:
frequency = y_train_3.value_counts()
total = sum(frequency)
class_weights = {i: count / total for i, count in enumerate(frequency)}

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], class_weight=class_weights)

Epoch 1/50
78/78 [==============================] - 2s 15ms/step - loss: 0.0180 - accuracy: 0.9815 - val_loss: 0.2061 - val_accuracy: 0.9357
Epoch 2/50
78/78 [==============================] - 1s 14ms/step - loss: 0.0111 - accuracy: 0.9867 - val_loss: 0.2110 - val_accuracy: 0.9212
Epoch 3/50
78/78 [==============================] - 1s 14ms/step - loss: 0.0143 - accuracy: 0.9847 - val_loss: 0.1202 - val_accuracy: 0.9550
Epoch 4/50
78/78 [==============================] - 1s 14ms/step - loss: 0.0059 - accuracy: 0.9944 - val_loss: 0.2856 - val_accuracy: 0.9164
Epoch 5/50
78/78 [==============================] - 1s 13ms/step - loss: 0.0096 - accuracy: 0.9903 - val_loss: 0.1374 - val_accuracy: 0.9502
Epoch 6/50
78/78 [==============================] - 1s 14ms/step - loss: 0.0106 - accuracy: 0.9887 - val_loss: 0.2151 - val_accuracy: 0.9341
Epoch 7/50
78/78 [==============================] - 1s 15ms/step - loss: 0.0072 - accuracy: 0.9932 - val_loss: 0.2371 - val_accuracy: 0.9357
Epoch 8/50
78

In [10]:
best_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
best_model.fit(X_train_pp_32, y_train_3, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
99/99 [==============================] - 3s 16ms/step - loss: 0.7314 - accuracy: 0.6718 - val_loss: 0.5140 - val_accuracy: 0.7497
Epoch 2/50
99/99 [==============================] - 1s 14ms/step - loss: 0.5246 - accuracy: 0.7575 - val_loss: 0.4636 - val_accuracy: 0.7624
Epoch 3/50
99/99 [==============================] - 1s 13ms/step - loss: 0.4605 - accuracy: 0.7753 - val_loss: 0.4136 - val_accuracy: 0.7967
Epoch 4/50
99/99 [==============================] - 1s 12ms/step - loss: 0.4124 - accuracy: 0.7887 - val_loss: 0.4665 - val_accuracy: 0.7598
Epoch 5/50
99/99 [==============================] - 1s 13ms/step - loss: 0.3817 - accuracy: 0.8122 - val_loss: 0.3425 - val_accuracy: 0.8463
Epoch 6/50
99/99 [==============================] - 1s 12ms/step - loss: 0.3259 - accuracy: 0.8481 - val_loss: 0.3021 - val_accuracy: 0.8717
Epoch 7/50
99/99 [==============================] - 1s 13ms/step - loss: 0.2519 - accuracy: 0.8888 - val_loss: 0.2792 - val_accuracy: 0.8679
Epoch 8/50
99

In [11]:
X_test_pp_32 = X_test_pp.apply(cv2.resize, dsize=(32, 32))
X_test_pp_32 = np.stack(X_test_pp_32.values)

best_model.evaluate(X_test_pp_32, y_test_3)

7/7 [==============================] - 0s 4ms/step - loss: 0.0620 - accuracy: 0.9712


[0.062014140188694, 0.9711538553237915]

In [57]:
best_model.save('models/best_category_model.keras')

## 숫자 모델

In [58]:
# reduce X_train, X_train_pp, X_train_pp_br to cases where df['category'] == 'number'
X_train_num = X_train[y_train['category'] == 'number']
X_train_pp_num = X_train_pp[y_train['category'] == 'number']

y_train_num = y_train[y_train['category'] == 'number']
y_train_num = pd.get_dummies(y_train_num['target'])
y_train_num = y_train_num[[str(i) for i in range(10)]]
X_train_num.shape, X_train_pp_num.shape, y_train_num.shape

((750,), (750,), (750, 10))

In [64]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

X_train_num_32 = X_train_num.apply(lambda x: s_preprocess_img(x, img_size=(32, 32)))
X_train_num_32 = np.stack(X_train_num_32.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_num_32, y_train_num, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
19/19 [==============================] - 1s 22ms/step - loss: 2.3255 - accuracy: 0.1033 - val_loss: 2.3168 - val_accuracy: 0.0733
Epoch 2/50
19/19 [==============================] - 0s 14ms/step - loss: 2.3072 - accuracy: 0.0750 - val_loss: 2.3083 - val_accuracy: 0.0400
Epoch 3/50
19/19 [==============================] - 0s 14ms/step - loss: 2.3010 - accuracy: 0.1117 - val_loss: 2.3069 - val_accuracy: 0.0867
Epoch 4/50
19/19 [==============================] - 0s 14ms/step - loss: 2.3013 - accuracy: 0.1100 - val_loss: 2.3066 - val_accuracy: 0.0600
Epoch 5/50
19/19 [==============================] - 0s 14ms/step - loss: 2.2989 - accuracy: 0.1183 - val_loss: 2.3086 - val_accuracy: 0.0400
Epoch 6/50
19/19 [==============================] - 0s 14ms/step - loss: 2.2951 - accuracy: 0.1250 - val_loss: 2.3089 - val_accuracy: 0.0867
Epoch 7/50
19/19 [==============================] - 0s 13ms/step - loss: 2.2773 - accuracy: 0.1433 - val_loss: 2.3187 - val_accuracy: 0.0533
Epoch 8/50
19

In [65]:
X_train_num_pp_32 = X_train_pp_num.apply(lambda x: cv2.resize(x, dsize=(32, 32)))
X_train_num_pp_32 = np.stack(X_train_num_pp_32.values)

model.fit(X_train_num_pp_32, y_train_num, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
19/19 [==============================] - 0s 19ms/step - loss: 2.2128 - accuracy: 0.1500 - val_loss: 2.0073 - val_accuracy: 0.2467
Epoch 2/50
19/19 [==============================] - 0s 13ms/step - loss: 1.9467 - accuracy: 0.2117 - val_loss: 1.8996 - val_accuracy: 0.1867
Epoch 3/50
19/19 [==============================] - 0s 14ms/step - loss: 1.8419 - accuracy: 0.2750 - val_loss: 1.8522 - val_accuracy: 0.2867
Epoch 4/50
19/19 [==============================] - 0s 14ms/step - loss: 1.8252 - accuracy: 0.2833 - val_loss: 1.8167 - val_accuracy: 0.2667
Epoch 5/50
19/19 [==============================] - 0s 13ms/step - loss: 1.6980 - accuracy: 0.3433 - val_loss: 1.7607 - val_accuracy: 0.3200
Epoch 6/50
19/19 [==============================] - 0s 14ms/step - loss: 1.6486 - accuracy: 0.3600 - val_loss: 1.7394 - val_accuracy: 0.3533
Epoch 7/50
19/19 [==============================] - 0s 14ms/step - loss: 1.5712 - accuracy: 0.4217 - val_loss: 1.6813 - val_accuracy: 0.3333
Epoch 8/50
19

In [66]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

X_train_num_64 = X_train_num.apply(lambda x: s_preprocess_img(x, img_size=(64, 64)))
X_train_num_64 = np.stack(X_train_num_64.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_num_64, y_train_num, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
19/19 [==============================] - 2s 61ms/step - loss: 2.3163 - accuracy: 0.0933 - val_loss: 2.3050 - val_accuracy: 0.0733
Epoch 2/50
19/19 [==============================] - 1s 58ms/step - loss: 2.3027 - accuracy: 0.1067 - val_loss: 2.3052 - val_accuracy: 0.0733
Epoch 3/50
19/19 [==============================] - 1s 52ms/step - loss: 2.2972 - accuracy: 0.1183 - val_loss: 2.3051 - val_accuracy: 0.0533
Epoch 4/50
19/19 [==============================] - 1s 51ms/step - loss: 2.2793 - accuracy: 0.1200 - val_loss: 2.3151 - val_accuracy: 0.0867
Epoch 5/50
19/19 [==============================] - 1s 52ms/step - loss: 2.2176 - accuracy: 0.1550 - val_loss: 2.2476 - val_accuracy: 0.1400
Epoch 6/50
19/19 [==============================] - 1s 52ms/step - loss: 2.0661 - accuracy: 0.2517 - val_loss: 2.1646 - val_accuracy: 0.1667
Epoch 7/50
19/19 [==============================] - 1s 52ms/step - loss: 1.9016 - accuracy: 0.3067 - val_loss: 1.9711 - val_accuracy: 0.3000
Epoch 8/50
19

In [72]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

X_train_pp_num_64 = X_train_pp_num.apply(lambda x: cv2.resize(x, dsize=(64, 64)))
X_train_pp_num_64 = np.stack(X_train_pp_num_64.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_num_64, y_train_num, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
19/19 [==============================] - 2s 66ms/step - loss: 2.3375 - accuracy: 0.1033 - val_loss: 2.3080 - val_accuracy: 0.1067
Epoch 2/50
19/19 [==============================] - 1s 60ms/step - loss: 2.3022 - accuracy: 0.0883 - val_loss: 2.3082 - val_accuracy: 0.0733
Epoch 3/50
19/19 [==============================] - 1s 63ms/step - loss: 2.3015 - accuracy: 0.1117 - val_loss: 2.3034 - val_accuracy: 0.0933
Epoch 4/50
19/19 [==============================] - 1s 60ms/step - loss: 2.2961 - accuracy: 0.1067 - val_loss: 2.2946 - val_accuracy: 0.0733
Epoch 5/50
19/19 [==============================] - 1s 59ms/step - loss: 2.2519 - accuracy: 0.1300 - val_loss: 2.1639 - val_accuracy: 0.1933
Epoch 6/50
19/19 [==============================] - 1s 61ms/step - loss: 2.0647 - accuracy: 0.2050 - val_loss: 1.9748 - val_accuracy: 0.1933
Epoch 7/50
19/19 [==============================] - 1s 58ms/step - loss: 1.9384 - accuracy: 0.2017 - val_loss: 1.9125 - val_accuracy: 0.2000
Epoch 8/50
19

In [74]:
best_number_model = model

X_test_pp_num = X_test_pp[y_test['category'] == 'number']
X_test_pp_num_64 = X_test_pp_num.apply(lambda x: cv2.resize(x, dsize=(64, 64)))
X_test_pp_num_64 = np.stack(X_test_pp_num_64.values)

y_test_num = y_test[y_test['category'] == 'number']
y_test_num = pd.get_dummies(y_test_num['target'])
y_test_num = y_test_num[[str(i) for i in range(10)]]

best_number_model.evaluate(X_test_pp_num_64, y_test_num)

8/8 [==============================] - 0s 14ms/step - loss: 0.0839 - accuracy: 0.9840


[0.08388721197843552, 0.984000027179718]

In [76]:
best_number_model.save('models/best_number_model.keras')

## 알파벳 모델

In [4]:
X_train_pp_alpha = X_train_pp[y_train['category'] == 'alphabet']

y_train_alpha = y_train[y_train['category'] == 'alphabet']
y_train_alpha = pd.get_dummies(y_train_alpha['target'])
y_train_alpha = y_train_alpha[[chr(i) for i in range(ord('A'), ord('Z')+1)]]
X_train_pp_alpha.shape, y_train_alpha.shape

((1965,), (1965, 26))

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(26, activation='softmax')
])

X_train_pp_alpha_64 = X_train_pp_alpha.apply(lambda x: cv2.resize(x, dsize=(64, 64)))
X_train_pp_alpha_64 = np.stack(X_train_pp_alpha_64.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_alpha_64, y_train_alpha, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
50/50 [==============================] - 4s 61ms/step - loss: 3.2635 - accuracy: 0.0312 - val_loss: 3.2578 - val_accuracy: 0.0662
Epoch 2/50
50/50 [==============================] - 3s 58ms/step - loss: 3.2312 - accuracy: 0.0585 - val_loss: 3.0896 - val_accuracy: 0.0840
Epoch 3/50
50/50 [==============================] - 3s 60ms/step - loss: 2.8616 - accuracy: 0.1215 - val_loss: 2.5186 - val_accuracy: 0.2137
Epoch 4/50
50/50 [==============================] - 3s 59ms/step - loss: 2.3167 - accuracy: 0.2697 - val_loss: 2.0047 - val_accuracy: 0.3486
Epoch 5/50
50/50 [==============================] - 3s 59ms/step - loss: 1.8749 - accuracy: 0.3906 - val_loss: 1.8180 - val_accuracy: 0.4224
Epoch 6/50
50/50 [==============================] - 3s 59ms/step - loss: 1.5863 - accuracy: 0.4701 - val_loss: 1.6561 - val_accuracy: 0.4249
Epoch 7/50
50/50 [==============================] - 3s 58ms/step - loss: 1.3970 - accuracy: 0.5254 - val_loss: 1.4500 - val_accuracy: 0.4987
Epoch 8/50
50

In [14]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(26, activation='softmax')
])

X_train_pp_alpha_32 = X_train_pp_alpha.apply(lambda x: cv2.resize(x, dsize=(32, 32)))
X_train_pp_alpha_32 = np.stack(X_train_pp_alpha_32.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_alpha_32, y_train_alpha, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
50/50 [==============================] - 2s 17ms/step - loss: 3.2656 - accuracy: 0.0344 - val_loss: 3.2591 - val_accuracy: 0.0356
Epoch 2/100
50/50 [==============================] - 1s 13ms/step - loss: 3.2585 - accuracy: 0.0382 - val_loss: 3.2607 - val_accuracy: 0.0305
Epoch 3/100
50/50 [==============================] - 1s 14ms/step - loss: 3.2547 - accuracy: 0.0496 - val_loss: 3.2430 - val_accuracy: 0.0662
Epoch 4/100
50/50 [==============================] - 1s 13ms/step - loss: 3.1369 - accuracy: 0.0891 - val_loss: 2.8983 - val_accuracy: 0.1450
Epoch 5/100
50/50 [==============================] - 1s 13ms/step - loss: 2.6053 - accuracy: 0.1851 - val_loss: 2.3090 - val_accuracy: 0.2239
Epoch 6/100
50/50 [==============================] - 1s 13ms/step - loss: 2.1547 - accuracy: 0.2856 - val_loss: 2.0832 - val_accuracy: 0.2824
Epoch 7/100
50/50 [==============================] - 1s 13ms/step - loss: 1.9345 - accuracy: 0.3416 - val_loss: 1.9873 - val_accuracy: 0.3181
Epoch 

In [91]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    horizontal_flip=False,   
    vertical_flip=False,   
    zoom_range=0.2,      
    shear_range=0.1,     
    fill_mode='nearest'      
)

augmented_data = datagen.flow(X_train_pp_alpha_32, y_train_alpha, batch_size=32)

model.fit(augmented_data, epochs=15, steps_per_epoch=len(X_train_pp_alpha_32) / 32)

Epoch 1/15
61/61 [==============================] - 1s 23ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 2/15
61/61 [==============================] - 2s 29ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 3/15
61/61 [==============================] - 2s 27ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 4/15
61/61 [==============================] - 2s 24ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 5/15
61/61 [==============================] - 2s 25ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 6/15
61/61 [==============================] - 1s 23ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 7/15
61/61 [==============================] - 1s 22ms/step - loss: 3.2582 - accuracy: 0.0422
Epoch 8/15
61/61 [==============================] - 1s 23ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 9/15
61/61 [==============================] - 1s 23ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 10/15
61/61 [==============================] - 1s 24ms/step - loss: 3.2581 - accuracy: 0.0422
Epoch 11/

In [92]:
model.evaluate(X_train_pp_alpha_32, y_train_alpha)

62/62 [==============================] - 0s 5ms/step - loss: 3.2578 - accuracy: 0.0422


[3.2577617168426514, 0.04223918542265892]

In [93]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(26, activation='softmax')
])

X_train_pp_alpha_224 = np.stack(X_train_pp_alpha.values)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pp_alpha_224, y_train_alpha, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
50/50 [==============================] - 48s 948ms/step - loss: 3.3453 - accuracy: 0.0541 - val_loss: 3.1848 - val_accuracy: 0.1145
Epoch 2/100
50/50 [==============================] - 47s 939ms/step - loss: 2.9907 - accuracy: 0.1323 - val_loss: 2.6706 - val_accuracy: 0.2443
Epoch 3/100
50/50 [==============================] - 49s 975ms/step - loss: 2.3886 - accuracy: 0.3060 - val_loss: 2.1592 - val_accuracy: 0.3486
Epoch 4/100
50/50 [==============================] - 49s 981ms/step - loss: 1.7563 - accuracy: 0.4523 - val_loss: 1.4774 - val_accuracy: 0.5700
Epoch 5/100
50/50 [==============================] - 51s 1s/step - loss: 1.1872 - accuracy: 0.6291 - val_loss: 0.9328 - val_accuracy: 0.7405
Epoch 6/100
50/50 [==============================] - 50s 1s/step - loss: 0.7573 - accuracy: 0.7570 - val_loss: 0.6872 - val_accuracy: 0.8397
Epoch 7/100
50/50 [==============================] - 51s 1s/step - loss: 0.5630 - accuracy: 0.8270 - val_loss: 0.5984 - val_accuracy: 0.8626
E

In [15]:
best_alphabet_model = model
best_alphabet_model.save('models/best_alphabet_model.keras')

In [18]:
X_test_alpha_pp = X_test_pp[y_test['category'] == 'alphabet']
X_test_alpha_pp_32 = X_test_alpha_pp.apply(lambda x: cv2.resize(x, dsize=(32, 32)))
X_test_alpha_pp_32 = np.stack(X_test_alpha_pp_32.values)

y_test_alpha = y_test[y_test['category'] == 'alphabet']
y_test_alpha = pd.get_dummies(y_test_alpha['target'])
y_test_alpha = y_test_alpha[[chr(i) for i in range(ord('A'), ord('Z')+1)]]

best_alphabet_model.evaluate(X_test_alpha_pp_32, y_test_alpha)

21/21 [==============================] - 0s 5ms/step - loss: 0.6098 - accuracy: 0.8931


[0.6097856163978577, 0.8931297659873962]